In [1]:
using Pkg
Pkg.activate("EnvFerrite")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


# Electrical Impedance Tomography with TV Regularization

## Introduction

In this example we give a basic implementation of of the real valued Calderon problem relevant to Electrical Impedance Tomography. We will generate data and afterwards solve the inverse problem with a numerical solver and implement TV regularization.  

### Forward EIT:
Given a conductivity $\gamma: \Omega\subset\mathbb{R}^2 \rightarrow \mathbb{R}_{+}$ our solution $u\in H^1(\Omega,\mathbb{R}^2)$ (2D case) has to confirm to the equation:


$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \forall x \in \Omega $$

For that equation we will choose a set of electrical current patterns 

$$ g_1, ..., g_n \in H^{-\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$

such that:  

$$ \int_{\partial\Omega}g_i \, d\partial\Omega = 0 $$

to inject into the material via Neumann boundary conditions:

$$ \gamma\frac{\partial u_i}{\partial n} =g_i \quad \forall x \in \partial\Omega $$

In order to get the corresponding voltages 
$$ f_1, ..., f_n \in H^{\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$
we will measure the corresponding voltage as:
$$ f_i := u_i|_{\partial\Omega} $$

with those boundary pairs $ (f_1,g_1), ... , (f_n,g_n) $ we now have an approximation of the Dirichlet to Neumann map:
$$ \Lambda_\gamma: H^{\frac{1}{2}}(\partial \Omega)\rightarrow H^{-\frac{1}{2}}(\partial \Omega) $$
This is called forward EIT since we just approximated the map:
$$ \gamma \rightarrow  \Lambda_\gamma $$

However real Electrical Impedance Tomography requires us to solve an **inverse Problem** where we have to reconstruct:
$$ \Lambda_\gamma \rightarrow  \gamma $$
for our approximation of $\Lambda_\gamma$ given by voltage-current boundary pairs $(f_i,g_i)$.

### Weak formulation
Given the strong formulation:
$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \text{with Neumann BC:}\quad \gamma\frac{\partial u}{\partial n} = g $$

The weak formulation is:
$$ \int\limits_\Omega \gamma \nabla(u)\cdot\nabla(v) \, d \Omega = \int\limits_{\partial\Omega} g \,v \,d\partial\Omega  \quad \forall v\in H^1(\Omega)$$

### Inverse EIT
We will use $\gamma$ to refer to the true underlying conductivity and $\sigma$ for our current conductivity guess.
We will choose the simplest minimization functional for optimization:
$$ J_i(u_i,\sigma) = \| f_i- u_i\|^2_{\mathcal{L}^2(\partial\Omega)} = \int_{\partial\Omega}(f_i-u_i)^2 \,d\partial\Omega $$
Such that our problem becomes:
$$ \underset{\sigma}{\min} \sum\limits_{i=1}^n J_i(u_i,\sigma) $$
such that:
$$ \nabla u_i\cdot(\sigma\nabla u_i) = 0\quad  \text{and Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i \quad \forall i\in\{1, ...,n\}$$
Given the problem our lagrangian becomes:
$$ \mathcal{L}(\sigma, u, \lambda) = \sum\limits_{i=1}^n\left( J_i(u_i,\sigma) + \langle \lambda_i, \nabla\cdot(\sigma\nabla u_i) \rangle_{\mathcal{L}^2(\Omega)}   \right) $$

from this we will use [Adjoint state methods](https://en.wikipedia.org/wiki/Adjoint_state_method#General_case) to calculate the gradient.
Without stating any of the steps of the derivation we end up with:
- State Equation (Variation with $\delta_\lambda$)
$$ \nabla \cdot(\sigma\nabla u_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i$$

- Adjoint Equation (Variation with $\delta_u$)
$$ \nabla \cdot(\sigma\nabla\lambda_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial \lambda_i}{\partial n} = 2(u_i-f_i)$$
- Functional Derivative (Variation with $\delta_\sigma$)
$$ \delta_i\sigma = -\nabla u_i \cdot \nabla \lambda_i $$

#### Total Variation (TV) regularization
for real world examples and numerical stability we have to assume that our system contains some noise, like $f = f_{true} + \epsilon$. Since the inverse EIT problem is highly ill conditioned we have to consider regularization.


Because $|\nabla\sigma|^2$ is non-differentiable when $\nabla\sigma=0$, we use
$$
\mathcal{R}_{TV}(\sigma) = \int_\Omega \sqrt{|\nabla\sigma|^2+\eta}\, d\Omega,
$$
and the gradient:
$$ \delta_\sigma\mathcal{R}= -\nabla\cdot \left(   \frac{\nabla\sigma}{\sqrt{|\nabla \sigma|^2+ \eta}} \right) $$
with a small $\eta$ to revent division by zero.
This is a $L^2$ projection that requires us to sove the weak form.
##### Weak formulation of TV Regularizer:
$$ \int_\Omega wv \,d\Omega = \int_\Omega \frac{\nabla(\sigma)}{\sqrt{|\nabla\sigma|^2+\eta}}\cdot\nabla(v)\, d\Omega \quad \forall v\in FESpace$$


### Full reconstruction Algorithm (Conceptual)
+ Preallocate Massmatrix M and L^2 projector (Cholesky factorization)
+ Start with conductivity guess $\sigma_0$ (In our case: $\sigma_0(x) = 1.0$)
+ Preallocate & initialize Conjugate Gradient(CG) solver for $u_1, ...,u_n.\lambda_1,.., \lambda_n,w$.
+ Repeat till tolerance is reached or other stopping condition:
    + From $\sigma_t$ assemble stiffness matrix $K_{\sigma_t}$
    + for all $i = 1, ...,n$ (in parallel)
        + Calculate $u_i$ (State equation) as well as the $\mathcal{L}^2(\partial\Omega)$ error: $\delta u_i$
        + Calculate $\lambda_i$ (Adjoint equation)
        + Calculate $\delta\sigma_i$ (Functional derivative)
    + Calculate TV Regularization gradient and error.
    + Update $\sigma_{t+1} = \sigma_t +\beta\, \delta_{TV}\sigma + \sum_{i=1}^n \alpha_i\,\delta_i\sigma $ (with Gauss-Newton with Levenberg-Marquardt).

## Implementation
### Preliminaries
Obvious Imports:

In [2]:
using Ferrite
using SparseArrays
using LinearAlgebra
using Revise
using Interpolations
using Plots
using Statistics
using IterativeSolvers

For simplicity we will use a quadratic grid with quadrilateral elements. We are using Quadrilaterals for now:

In [3]:
grid = generate_grid(Quadrilateral, (32, 32));
dim = Ferrite.getspatialdim(grid)
order = 1


ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(2)
qr_face = FacetQuadratureRule{RefQuadrilateral}(2)
cellvalues = CellValues(qr, ip)
facetvalues = FacetValues(qr_face, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)



∂Ω = union(getfacetset.((grid,), ["left", "top", "right", "bottom"])...)
length(∂Ω)

128

For later use we will assemble and cholesky decompose the mass matrix once.

In [4]:
# This is supposed to be: ∫(u*v)dΩ
function assemble_M(cellvalues::CellValues,dh::DofHandler)
    M = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Me = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Me, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)      
            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    φⱼ = shape_value(cellvalues, q_point, j)
                    Me[i,j] += φᵢ * φⱼ * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Me)
    end
    return M, cholesky(M)
end   

M, MC = assemble_M(cellvalues,dh)

(sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  1054, 1055, 1056, 1087, 1088, 1089, 1055, 1056, 1088, 1089], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  1088, 1088, 1088, 1088, 1088, 1088, 1089, 1089, 1089, 1089], [0.0004340277777777777, 0.00021701388888888885, 0.00010850694444444441, 0.00021701388888888882, 0.00021701388888888885, 0.0008680555555555555, 0.0004340277777777777, 0.00010850694444444441, 0.00021701388888888893, 0.00010850694444444444  …  0.00010850694444444444, 0.0004340277777777777, 0.00010850694444444441, 0.00021701388888888882, 0.0008680555555555553, 0.00021701388888888882, 0.00010850694444444441, 0.00021701388888888882, 0.00021701388888888882, 0.00043402777777777765], 1089, 1089), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  20895
nnz:     20895
success: true
)

Furthermore we want to know which entries of the force vector correspond to the boundary:
We will get:
- the count of nonzero entries in the force vector
- the position of non zero entries
- a function "up" to cast a vector of length of boundary dofs into the length of the force vector
- a function "down" to cast a vector into the length of the dofs of the force vector that lay on the boundary.

In [5]:
function produce_nonzero_positions(v, atol=1e-8, rtol=1e-5)
    approx_zero(x; atol=atol, rtol=rtol) = isapprox(x, 0; atol=atol, rtol=rtol)
    non_zero_count = count(x -> !approx_zero(x), v)
    non_zero_positions = zeros(Int, non_zero_count)
    non_zero_indices = findall(x -> !approx_zero(x), v)
    g_down = (x) -> x[non_zero_indices]
    g_up = (x) -> begin
        v = zeros(eltype(x), length(v))
        v[non_zero_indices] = x
        return v
    end
    return non_zero_count, non_zero_positions, g_down, g_up
end
function produce_nonzero_positions(facetvalues::FacetValues, dh::DofHandler, ∂Ω)
    f = zeros(ndofs(dh))
        for facet in FacetIterator(dh, ∂Ω)
        fe = zeros(ndofs_per_cell(dh))
        reinit!(facetvalues, facet)
        for q_point in 1:getnquadpoints(facetvalues)
            dΓ = getdetJdV(facetvalues, q_point)            
            for i in 1:getnbasefunctions(facetvalues)
                δu = shape_value(facetvalues, q_point, i)
                fe[i] += δu * dΓ
            end
        end
        assemble!(f, celldofs(facet), fe)
    end
     nzc, nzpos, g_down, g_up = produce_nonzero_positions(f)
     return  nzc, nzpos, g_down, g_up, f
end

nzc,nzpos, down, up = produce_nonzero_positions(facetvalues, dh,∂Ω)
@assert nzc == length(∂Ω)  # This is not true in Gridap.jl

In [6]:
## Sanity check:
# I have never questioned the assumption that up∘down == id and down∘up == id. Maybe I should check this.
test_vec = [i for i in 1:nzc]
@assert down(up(test_vec)) == test_vec
@assert up(down(up(test_vec))) == up(test_vec)

### Data generation

Now we will make up some conductivity. As well as some current patterns:

In [7]:
conductivity  = (x) -> 1.1 + sin(x[1]) * cos(x[2])

#22 (generic function with 1 method)

For later we want to project that function unto Q1 FE space for that we want to assemble the coefficients in the FESpace:

In [8]:
function assemble_function_vector(cellvalues::CellValues, dh::DofHandler, f, M_cholesky)
    F = zeros(ndofs(dh))
    n_basefuncs = getnbasefunctions(cellvalues)
    Fe = zeros(n_basefuncs)
    cdofs = zeros(Int, n_basefuncs)

    for cell in CellIterator(dh)
        fill!(Fe, 0.0)
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        cdofs = celldofs(cell)
        for q in 1:getnquadpoints(cellvalues)
            x_q = spatial_coordinate(cellvalues, q, coords)
            f_val = f(x_q)
            dΩ = getdetJdV(cellvalues, q)

            for i in 1:n_basefuncs
                Fe[i] += f_val * shape_value(cellvalues, q, i) * dΩ
            end
        end  
        assemble!(F, cdofs,Fe)
    end
    return M_cholesky \ F
end
cond_vec = assemble_function_vector(cellvalues, dh, conductivity, M)

1089-element Vector{Float64}:
 0.6450535642159253
 0.6641859150714466
 0.6226468097664015
 0.601690881462597
 0.685021948586164
 0.645468805231958
 0.707477970024696
 0.6700651958589253
 0.7314669135515499
 0.6963406172344299
 ⋮
 1.3592050021692093
 1.3883340781632245
 1.416337215072183
 1.4431050644646248
 1.4685330864484494
 1.492522029975305
 1.5149780514138347
 1.5358140849285546
 1.5549464357840737

In [9]:
# This function assembles the stiffness matrix from a given vector.
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ::AbstractVector)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            γe = γ[celldofs(cell)] # (Edit) Could be done more efficiently by copying into preallocated array
            σ = function_value(cellvalues, q_point, γe)
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q_point, i)
                #u = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += σ* (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_from_vec = assemble_K(cellvalues, dh,cond_vec)

1089×1089 SparseMatrixCSC{Float64, Int64} with 9409 stored entries:
⎡⠻⣦⡸⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠲⢮⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⎦

In [10]:
# This is matrix assembly on a function. How do I do it if the conductivity is given as a coefficient vector for Q1 FE Space?
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)
            x = spatial_coordinate(cellvalues, q, getcoordinates(cell))
            σ = γ(x)
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q, j)
                    Ke[i, j] += σ * (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_true = assemble_K(cellvalues, dh, conductivity)
K_true_LU = lu(K_true)

SparseArrays.UMFPACK.UmfpackLU{Float64, Int64}
L factor:
1089×1089 SparseMatrixCSC{Float64, Int64} with 20895 stored entries:
⎡⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠴⠿⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠤⠀⠀⠿⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⣐⣳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠨⣵⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠒⠛⠛⠛⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠐⠶⠀⠐⠶⠿⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠚⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠛⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣠⣤⢤⣤⣴⢾⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠀⠀⠀⠀⠉⠉⣳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢨⡷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠱⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠴⠾⠷⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣠⡄⣤⣤⡴⢶⣾⣷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢠⣄⣴⠒⠆⠀⠀⠰⣤⣴⣏⢛⠀⠀⠀⠐⠒⠒⢚⣯⢩⠒⠛⠃⠀⠀⠐⠶⢿⣷⣄⠀⠀⎥
⎣⣲⠆⣴⡏⠛⣧⣴⢾⡳⣿⠀⠉⠁⠀⠉⠛⠛⠧⠠⠤⠀⣤⣤⠀⠀⣤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⣿⣷⣄⎦
U factor:
1089

In [11]:
# Sanity check: (positive semidefinite self adjoint stiffness matrix)
@assert K_true == K_true'  # Not true in Gridap.jl
if ndofs(dh) < 500
    K_dense = Matrix(K_true)
    eig_min = minimum(eigvals(K_dense))
    @assert eig_min > -1e-14
    print("Smallest eigenvalue: ", eig_min)
end

Now we generate current patterns: We assume one current source and one current sink. Important is that it sums of to zero.
We generate the right hand side force vectors $g_1, ... g_n$ as an Matrix G and calculate $f_1, ..., f_n$

In [12]:
num_modes = Int64((nzc^2-nzc)//2)
G_small = zeros(nzc,num_modes)
G = zeros(ndofs(dh),num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        G_small[i,k] = 1.0
        G_small[j,k] = -1.0
        G[:,k] = up(G_small[:,k])
        k += 1
    end
end
F_big = K_true_LU \ G
F = zeros(nzc,num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        F[:,k] = down(F_big[:,k])
        k += 1
    end
end
col_means = mean(F, dims=1)
F .-= col_means

128×8128 Matrix{Float64}:
  2.28031     2.25774       2.8858     …   0.00896351   0.00471518
 -0.822377    0.128708      0.503204       0.00896068   0.00471448
  0.15127    -0.907189      0.708219       0.00896638   0.0047159
 -0.102286    0.0801673    -1.03945        0.00895235   0.00471241
 -0.0645602   0.030441     -0.242412       0.00893867   0.00470901
 -0.0399858   0.0134256    -0.147098   …   0.00891984   0.00470433
 -0.0299563   0.00458881   -0.0979698      0.00889602   0.0046984
 -0.0244935  -0.000326226  -0.074219       0.00886739   0.00469129
 -0.0212772  -0.00332372   -0.0605148      0.00883415   0.00468302
 -0.0192336  -0.00526549   -0.0519835      0.0087965    0.00467366
  ⋮                                    ⋱               
 -0.0134128  -0.0109905    -0.0283843      0.0240971    0.00839202
 -0.013424   -0.0109791    -0.0284286  …   0.0284299    0.00941065
 -0.0134336  -0.0109693    -0.0284667      0.0352115    0.0109726
 -0.0134417  -0.0109611    -0.0284985      0.04667

To be realistic we will add some noise:
We have to ensure that our noise has mean zero.

In [13]:
function mean_zero_noise(n::Int64, σ::Float64)
    out = σ * randn(n)
    mean = Statistics.mean(out)
    out .- mean
end

mean_zero_noise (generic function with 1 method)

To simplify things we can also do SVD:

In [14]:
# Implement SVD here later
# reduce the number of modes according to used SVD modes
function do_svd(F,G)
    Λ = F * G'
    num_modes = (size(Λ, 1) - 1)
    V,Σ,U = svd(Λ)
    U = U[:,1:num_modes]
    V = V[:,1:num_modes]
    col_means = mean(U, dims=1)
    U .-= col_means
    col_means = mean(V, dims=1)
    V .-= col_means
    # Not unimportant:
    V = V * diagm(Σ[1:num_modes]) 
    return V, Σ, U, Λ, num_modes
end
F, Σ, G_small, Λ, num_modes = do_svd(F,G_small)
# Apply singular Values:
G = zeros(ndofs(dh),num_modes)
F_big = copy(G)
for i in 1:num_modes
    G[:,i] = up(G_small[:,i])
    F_big[:,i] = up(F[:,i])
end

now that we have done SVD on the original choice of modes we have 
- Truncation of SVD modes, thus regularization
- Averaging of noise over multiple measurements
- Elimination of any unncessary number of nodes we choose before.

Here we define a struct where we save and preallocate all the necessary information for the solver step.

In [15]:
# Implement a sanity check if the two matrices assembled from the function and the vector are roughly the same (use relatively coarse ≈ )
Matrix_norm = norm(K_from_vec - K_true)
println("Norm of Matrix difference: ",Matrix_norm)
@assert Matrix_norm < 20.0

g_test = G[:,1]
f_test_true = K_true \ g_test
f_test_vec = K_from_vec \g_test
vector_norm = norm(f_test_true - f_test_vec)
println("norm of difference of first SVD mode: " ,vector_norm)
@assert vector_norm < 10.0

Norm of Matrix difference: 8.536307354525758e-6
norm of difference of first SVD mode: 12.060954332454587


AssertionError: AssertionError: vector_norm < 10.0

In [16]:
mutable struct EITMode
    u::AbstractVector
    λ::AbstractVector
    δσ::AbstractVector
    f::AbstractVector
    g::AbstractVector
    rhs::AbstractVector
    error::Float64
    length::Int64
    m::Int64
end
function EITMode(g::AbstractVector, f::AbstractVector)
    L = length(g)
    M = length(f)
    return EITMode(zeros(L), zeros(L), zeros(L), f, g, zeros(L), 0.0, L, M)
end
mode_dict = Dict{Int64,EITMode}()
for i in 1:num_modes
    mode_dict[i] = EITMode(G[:,i],F[:,i])
end

### Solving EIT

We will now assume a starting conductivity guess $\sigma_0(x) = 1.0 $

In [17]:
σ₀ = (x) -> 1.0

#24 (generic function with 1 method)

We would prefer to save $\sigma$ as a vector for use in FEM and also have a method to export each

In [18]:
# Project function here: 
σ = assemble_function_vector(cellvalues,dh, σ₀, MC)


1089-element Vector{Float64}:
 0.9999999999999998
 1.0
 0.9999999999999996
 1.0000000000000002
 0.9999999999999989
 1.0000000000000007
 1.0000000000000002
 0.9999999999999992
 0.9999999999999996
 1.0000000000000009
 ⋮
 1.0
 1.0000000000000004
 1.0
 1.0000000000000009
 0.9999999999999997
 1.0000000000000007
 0.9999999999999998
 1.0000000000000002
 0.9999999999999997

A prerequisite is that we can calculate the bilinear map: $\nabla(u)\cdot\nabla(\lambda)$


In [19]:
# Assemble right-hand side for the projection of ∇(u) ⋅ ∇(λ) onto the FE space.
# This computes rhs_i = ∫ (∇u ⋅ ∇λ) ϕ_i dΩ for each test function ϕ_i.
# Assuming u and λ are scalar fields in the same FE space.
# cellvalues should be CellScalarValues(qr, ip) where qr is QuadratureRule, ip is Interpolation.
function calculate_bilinear_map(a::AbstractVector, b::AbstractVector, cellvalues::CellValues, dh::DofHandler, M_cholesky)
    n = ndofs(dh)
    rhs = zeros(n)
    n_basefuncs = getnbasefunctions(cellvalues)
    qpoints = getnquadpoints(cellvalues)
    re = zeros(n_basefuncs)
    
    for cell in CellIterator(dh)
        dofs = celldofs(cell)
        reinit!(cellvalues, cell)
        fill!(re, 0.0)
        
        ae = a[dofs]
        be = b[dofs]
        
        for q in 1:qpoints
            dΩ = getdetJdV(cellvalues, q)
            
            ∇a_q = zero(Vec{2,Float64})
            ∇b_q = zero(Vec{2,Float64})
            
            for j in 1:n_basefuncs
                ∇ϕⱼ = shape_gradient(cellvalues, q, j)
                ∇a_q += ae[j] * ∇ϕⱼ
                ∇b_q += be[j] * ∇ϕⱼ
            end
            
            grad_dot_product = ∇a_q ⋅ ∇b_q
            
            for i in 1:n_basefuncs
                ϕᵢ = shape_value(cellvalues, q, i)
                re[i] += grad_dot_product * ϕᵢ * dΩ
            end
        end
        assemble!(rhs, dofs, re)
    end
    
    return M \ rhs
end

function calculate_bilinear_map!(out::AbstractVector , rhs::AbstractVector, a::AbstractVector, b::AbstractVector, cellvalues::CellValues, dh::DofHandler, M_cholesky)
    n = ndofs(dh)
    fill!(rhs,0.0)
    n_basefuncs = getnbasefunctions(cellvalues)
    qpoints = getnquadpoints(cellvalues)
    re = zeros(n_basefuncs)
    
    for cell in CellIterator(dh)
        dofs = celldofs(cell)
        reinit!(cellvalues, cell)
        fill!(re, 0.0)
        
        ae = a[dofs]
        be = b[dofs]
        
        for q in 1:qpoints
            dΩ = getdetJdV(cellvalues, q)
            
            ∇a_q = zero(Vec{2,Float64})
            ∇b_q = zero(Vec{2,Float64})
            
            for j in 1:n_basefuncs
                ∇ϕⱼ = shape_gradient(cellvalues, q, j)
                ∇a_q += ae[j] * ∇ϕⱼ
                ∇b_q += be[j] * ∇ϕⱼ
            end
            
            grad_dot_product = ∇a_q ⋅ ∇b_q
            
            for i in 1:n_basefuncs
                ϕᵢ = shape_value(cellvalues, q, i)
                re[i] += grad_dot_product * ϕᵢ * dΩ
            end
        end
        assemble!(rhs, dofs, re)
    end
    
    out = M \ rhs
    return out
end

calculate_bilinear_map! (generic function with 1 method)

With the given matrix and projector our we need to optimize for every mode $(f_i,g_i)$:

In [20]:
function state_adjoint_step!(mode::EITMode, K::AbstractMatrix, M, dh::DofHandler, cellvalues::CellValues, maxiter=500)
    cg!(mode.u,K, mode.g; maxiter = maxiter)
    b = down(mode.u)
    b = 2*(b - mode.f)
    b .-= Statistics.mean(b)
    cg!(mode.λ, K, up(b); maxiter = maxiter)
    mode.error = norm(b)
    # add calculation of ∇(u)⋅∇(λ) here once figured out
    mode.δσ = calculate_bilinear_map(mode.λ, mode.u, cellvalues, dh, M)    
    # Check whether this needs + or - as a sign.
end

function state_adjoint_step_initial!(mode::EITMode, K_LU, M, dh::DofHandler, cellvalues::CellValues)
    mode.u = K_LU \ mode.g
    b = down(mode.u)
    b = 2*(b - mode.f)
    b .-= Statistics.mean(b)
    mode.λ = K_LU \ up(b)
    mode.error = norm(b)
    # add calculation of ∇(u)⋅∇(λ) here once figured out
    mode.δσ = calculate_bilinear_map(mode.λ, mode.u, cellvalues, dh, M)    
    # Check whether this needs + or - as a sign.
end

state_adjoint_step_initial! (generic function with 1 method)

Additinal we need to assemble the TV regularizer. The required Mass matrix we already have asembled and ready to use. 


In [21]:
mutable struct TV
    δ::AbstractArray # Is supposed to hold the error
    rhs::AbstractArray # 
    err_vec::AbstractArray
    error::Float64
end
function TV(n::Int64)
    TV(zeros(n),zeros(n),zeros(n),0.0)
end

function calc_tv_step!(σ::AbstractVector{Float64}, tv::TV, dh::DofHandler, cellvalues::CellValues, M, η=1e-8)
    n = ndofs(dh)
    r = tv.rhs
    r = zeros(n)
    tv_error = 0.0

    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        dofs = celldofs(cell)
        σe = σ[dofs]

        re = zeros(length(dofs))
        # quadrature loop
        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)
            ∇ϕ = shape_gradient(cellvalues, q)   # size (dim, ndofs)
            ∇σ = ∇ϕ * σe                        # vector of length dim
            a = 1 / sqrt(dot(∇σ, ∇σ) + η)
            tv_error += sqrt(dot(∇σ, ∇σ) + η) * dΩ

            ϕvals = shape_function_values(cellvalues, q)
            for i in 1:length(dofs)
                ∇ϕi = ∇ϕ[:, i]
                re[i] += a * dot(∇σ, ∇ϕi) * dΩ
            end
        end
        assemble!(r, dofs, re)
    end

    tv.δ = M \ r          
    tv.error = tv_error
end
tv = TV(ndofs(dh))

TV([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0)

For finding suitable stepsizes we will use Gauss-Newton.
Since we want to avoid implementing a dense Hessian Matrix we use SVD to invert the Matrix.

In [22]:
function gauss_newton(J::Matrix{Float64}, r::Vector{Float64}; λ::Float64=1e-3)
    U, Σ, V = svd(J, full=false)
    n = length(Σ)
    Σ_damped = zeros(n)
    for i in 1:n
        Σ_damped[i] = Σ[i] / (Σ[i]^2 + λ) # Levenberg-Marquardt regularization
    end
    V * (Σ_damped .* (U' * r))
end
# Note: If we do SVD further up on the boundary pairs that we selected we can use truncated SVD to approximate/regularize the boundary operator Λ. Due to that (i.e. a highly reduced number of boundary pairs) SVD in Gauss-newton might become an efficient option. Else I want to avoid assembling a huge dense Hessian Matrix.

gauss_newton (generic function with 1 method)

Now that we have all the pieces we can assemble the full optimization step:

In [23]:
# note: If you want to use truncated SVD as regularization one can pass a smaller number than num_modes
function full_step!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV, dh::DofHandler, cellvalues::CellValues, do_TV::Bool =true, β::Float64 = 1e-5)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    if do_TV
        J = zeros(num_modes+1,ndofs(dh))
        r = zeros(num_modes+1)
        # Launch TV regularizer:
        tv_task = Threads.@spawn begin
            calc_TV_step!(σ,tv, dh,cellvalues,M)
        end
    else
        J = zeros(num_modes,ndofs(dh))
        r = zeros(num_modes)
    end
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M, dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in 1:num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    if do_TV
        # Fetch TV regularization
        fetch(tv_task)
        J[num_modes+1,:] = tv.δ
        r[num_modes+1] = β * tv.error 
    end    
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    σ .-= δσ
    σ .= max.(σ ,1e-12) # Ensure positivity
    return σ,δσ,r
end

full_step! (generic function with 3 methods)

In [24]:
function full_step_initial!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV, dh::DofHandler, cellvalues::CellValues)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    K_LU = lu(K)
    J = zeros(num_modes,ndofs(dh))
    r = zeros(num_modes)
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M, dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in 1:num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    println(δσ)
    σ .-= δσ
    σ .= max.(σ ,1e-12) # Ensure positivity
    return K,J,δσ,r
end

full_step_initial! (generic function with 1 method)

In [25]:
error  = norm(cond_vec - σ)
println("With error: ", error)

With error: 15.284988831024412


Let's run this optimization loop a few times:

In [26]:
result, ftime, bytes, gctime, memallocs = @timed begin
    full_step_initial!(MC, σ, mode_dict, num_modes, tv, dh, cellvalues)
end
println("Time: ", ftime, " seconds, Bytes: ", bytes, ", GC time: ", gctime, ", Memory allocations: ", memallocs)

[-0.00040678675893775034, -0.004838327402799584, -0.004117511149112741, -0.0005128978531992347, -0.0020047575569174716, 0.01436297179080164, -0.0025665719108181268, 0.006422266161323681, -0.0027763260320212034, 0.0003605936297533587, -0.0033841894373925546, -0.004095111108287325, -0.003321264910403851, -0.0060500965602266745, -0.003604487962392288, -0.0012953994705108707, -0.0031902618956787527, 0.0027219141734083673, -0.003394609528526532, 0.00585055088054237, -0.0030486506855315525, 0.005654350256458143, -0.003286953826450735, 0.006298555592566267, -0.002941729466211801, 0.0049154920629108645, -0.0032309147955901936, 0.0036246494777627887, -0.0030017368090115943, -0.00045751970771907386, -0.003375692406907203, -0.0029492286084867787, -0.003100541329106213, -0.0047562151079835425, -0.0032983734171745946, -0.0023248961260052873, -0.002839128476556118, 0.00011295042394287311, -0.002997214788478748, 0.0028024472445789626, -0.002721102399595614, 0.0005324577822288844, -0.00319416714424145

In [27]:
num_modes

127

In [28]:
error  = norm(cond_vec - σ)
    println("With error: ", error)

With error: 15.284774362232643


In [29]:

for i in 1:20
    print("Step ", i, " ")
    result, time, bytes, gctime, memallocs = @timed begin
        full_step!(MC, σ, mode_dict, 10, tv, dh, cellvalues, false)
    end
    #println("Time: ", time, " seconds, Bytes: ", bytes, ", GC time: ", gctime, ", Memory allocations: ", memallocs)
    error  = norm(cond_vec - σ)
    println("With error: ", error)
end


Step 1 With error: 15.284082077754466
Step 2 With error: 15.283411267462581
Step 3 With error: 15.28276202696734
Step 4 With error: 15.282134449176096
Step 5 With error: 15.281528624422766
Step 6 With error: 15.280944640603684
Step 7 With error: 15.280382583318449
Step 8 With error: 15.27984253601461
Step 9 With error: 15.279324580135137
Step 10 With error: 15.278828795267675
Step 11 With error: 15.278355259294706
Step 12 With error: 15.277904048543745
Step 13 With error: 15.277475237936828
Step 14 With error: 15.27706890113861
Step 15 With error: 15.276685110702473
Step 16 With error: 15.276323938214066
Step 17 With error: 15.27598545443185
Step 18 With error: 15.275669729424205
Step 19 With error: 15.275376832702754
Step 20 With error: 15.275106833351625


In [30]:
#mode_dict[1].δσ
σ

1089-element Vector{Float64}:
 1.0481279172914761
 1.025350688151798
 1.012071536995171
 1.0235110774092129
 1.0170344383550665
 0.9913998646005739
 1.0196717956980441
 1.0002206853271984
 1.0218723222862423
 1.0090378048657003
 ⋮
 1.0181866988118755
 1.0185463534945076
 1.0189871849662393
 1.0208018714815095
 1.0214497799331166
 1.0218258970302847
 1.0225279061232746
 1.0215702966527807
 1.0259213130371387

In [31]:
#σ,δσ,r = full_step!(MC, σ, mode_dict, num_modes, tv, dh, cellvalues, false)

In [32]:
# Test the above functions

## Plotting


In [33]:
# project is to grid and plot with Plots.jl
# I wanna use Plots.jl and not Makie.jl or similar because lateron i want to implement a NN on the grid as a regularizer using Lux.jl

In [34]:
# Project to dictionary (coordinate,value)
#PointEvalHandler(grid, points::AbstractVector{Vec{dim,T}})
# Put values from dictionary into Array.

This is our original and final reconstruction:

In [35]:
# Plot reconstruction and original with Plots.jl here